In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df_1_3 = pd.read_csv('results/main_table.tsv',  sep='\t')
df_4_bgpd_gbm = pd.read_csv('results/transfer_bgpd_gbm.tsv',  sep='\t')
df_4_gbm_bgpd = pd.read_csv('results/transfer_gbm_bgpd.tsv',  sep='\t')

In [2]:
df_1_3.head()

,PatientID,Preprocessing step,Dataset,"NN-Architecture (nn-Unet, UNETR)","Label (WT, ET, TC, GTV)",Dice,Hausdorff,Surface Dice 1mm,Surface Dice 3mm,Surface Dice 5mm,Epoch,Volume_estimated,Volume_intersection,Volume_true
0,TCGA-02-0011,1_reg,gbm,nn_unet,WT,0.290,39.000,0.264,NaN,NaN,300,25.748,22.114,126.783
1,TCGA-02-0011,1_reg,gbm,nn_unet,ET,0.138,39.000,0.218,NaN,NaN,300,2.983,22.114,33.488
2,TCGA-02-0011,1_reg,gbm,nn_unet,TC,0.030,34.300,0.133,NaN,NaN,300,9.525,22.114,81.619
3,TCGA-02-0033,1_reg,gbm,nn_unet,WT,0.242,39.000,0.261,NaN,NaN,300,19.781,14.996,104.407
4,TCGA-02-0033,1_reg,gbm,nn_unet,ET,0.265,26.094,0.330,NaN,NaN,300,6.324,14.996,31.703


In [3]:
## Table 4: 

reindex_list = ['1_reg', '2a_interp', '3a_atlas', '4a_resamp', '4b_n4', '4d_susan','6_hist','5_ss_shared']
rename_list = ['1. Inter modality', '2. Resampling', '3. Atlas', '4. Resampling to spacing', '4.a Bias', '4.b Denoising', '4.c Histogram', '4.d Skull']


In [4]:
df_1_3.Dataset.unique()

array(['gbm', 'bgpd', 'lgg'], dtype=object)

По каждой метрике проверяем на gbm, bgpd, lgg. Делаем для nnU-net и unetr.

In [5]:
#Table 4 DICE WT
import numpy as np
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()
epoch = 300
# to save space in the paper and not write 0.44 and write 44 instead (only for Dice and Surface Dice)
multiplier = 100


def get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier):
    
    # mean
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model ].groupby([ 'Preprocessing step'])[metric].mean(),3)*multiplier).round(0)
    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_mean = temp_series

    #std
    temp_series = (np.round(df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model].groupby([ 'Preprocessing step'])[metric].std(),3)*multiplier).round(0)

    temp_series = temp_series.reindex(reindex_list)
    temp_series.index = rename_list
    temp_series_std = temp_series

    temp_mean_std = pd.concat([temp_series_mean, temp_series_std], axis=1)
    temp_mean_std.columns = ['Mean', 'Std']
    temp_mean_std['Mean (STD)'] = temp_mean_std.apply(lambda x: str(x['Mean']) + ' (' + str(x['Std']) + ')', axis=1)
    return temp_mean_std['Mean (STD)'] 

print('DICE scores')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier))
        print( )

DICE scores
=======================NEW MODEL=======================
gbm nn_unet Dice
1. Inter modality           44.0 (28.0)
2. Resampling               85.0 (11.0)
3. Atlas                    85.0 (11.0)
4. Resampling to spacing    85.0 (12.0)
4.a Bias                    82.0 (13.0)
4.b Denoising               84.0 (12.0)
4.c Histogram               83.0 (16.0)
4.d Skull                   87.0 (11.0)
Name: Mean (STD), dtype: object

bgpd nn_unet Dice
1. Inter modality           36.0 (29.0)
2. Resampling               73.0 (19.0)
3. Atlas                    75.0 (16.0)
4. Resampling to spacing    74.0 (18.0)
4.a Bias                    75.0 (17.0)
4.b Denoising               74.0 (17.0)
4.c Histogram               75.0 (16.0)
4.d Skull                   76.0 (14.0)
Name: Mean (STD), dtype: object

lgg nn_unet Dice
1. Inter modality           67.0 (27.0)
2. Resampling               72.0 (24.0)
3. Atlas                    71.0 (25.0)
4. Resampling to spacing    70.0 (25.0)
4.a Bias      

In [6]:
# Appending Table  Hausdorff WT
metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
models = df_1_3['NN-Architecture (nn-Unet, UNETR)'].unique()

#get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=1)

print('Hausdorff scores')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=1))
        print( )

Hausdorff scores
=======================NEW MODEL=======================
gbm nn_unet Hausdorff
1. Inter modality           31.0 (21.0)
2. Resampling               15.0 (21.0)
3. Atlas                    10.0 (11.0)
4. Resampling to spacing    13.0 (20.0)
4.a Bias                    14.0 (22.0)
4.b Denoising               11.0 (16.0)
4.c Histogram               13.0 (18.0)
4.d Skull                   10.0 (14.0)
Name: Mean (STD), dtype: object

bgpd nn_unet Hausdorff
1. Inter modality             nan (nan)
2. Resampling               26.0 (33.0)
3. Atlas                    22.0 (27.0)
4. Resampling to spacing    23.0 (34.0)
4.a Bias                      nan (nan)
4.b Denoising                 nan (nan)
4.c Histogram                 nan (nan)
4.d Skull                   15.0 (17.0)
Name: Mean (STD), dtype: object

lgg nn_unet Hausdorff
1. Inter modality           35.0 (36.0)
2. Resampling               29.0 (32.0)
3. Atlas                    27.0 (35.0)
4. Resampling to spacing    38.0 (

In [7]:
# Appending Table  Surface Dice WT
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'

print('Surface Dice WT')
for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print(data, model, metric)
        print(get_column_from_pivot_table(df_1_3, metric, data, label, model, epoch, multiplier=100))
        print( )

Surface Dice WT
=======================NEW MODEL=======================
gbm nn_unet Surface Dice 1mm
1. Inter modality           27.0 (15.0)
2. Resampling               65.0 (20.0)
3. Atlas                    63.0 (20.0)
4. Resampling to spacing    65.0 (20.0)
4.a Bias                    60.0 (22.0)
4.b Denoising               64.0 (21.0)
4.c Histogram               63.0 (22.0)
4.d Skull                   69.0 (20.0)
Name: Mean (STD), dtype: object

bgpd nn_unet Surface Dice 1mm
1. Inter modality           20.0 (14.0)
2. Resampling               41.0 (18.0)
3. Atlas                    44.0 (17.0)
4. Resampling to spacing    43.0 (18.0)
4.a Bias                    44.0 (19.0)
4.b Denoising               42.0 (18.0)
4.c Histogram               43.0 (18.0)
4.d Skull                   45.0 (19.0)
Name: Mean (STD), dtype: object

lgg nn_unet Surface Dice 1mm
1. Inter modality           41.0 (24.0)
2. Resampling               47.0 (24.0)
3. Atlas                    46.0 (26.0)
4. Resampling 

In [8]:
### STATISTICAL SIGNIFICANCE

In [9]:
import scipy
from statsmodels.stats.weightstats import ttost_ind, ttost_paired

metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300


preproc_1 = "4a_resamp"
for key in reindex_list:
    preproc_2 = key
    
    if key != "4a_resamp":

        data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
        data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
        
        #assert if nan
        assert data_1.isnull().sum() == 0
        assert data_2.isnull().sum() == 0
        
        data_1 = np.array(data_1)
        data_2 = np.array(data_2)
        
        ttost = ttost_paired(data_1, data_2, -data_2.std()/4, data_2.std()/4)[0]/7
        ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

        # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
        if (ttest_ < 0.05)&(ttost > 0.05):
            print(preproc_2, (np.round(data_1.mean(),3)*1).round(0).astype(int), '(',
              (np.round(data_1.std(),3)*1).round(0).astype(int), ')',
              ' vs ',\
              (np.round(data_2.mean(),3)*1).round(0).astype(int), '(' ,
              (np.round(data_2.std(),3)*1).round(0).astype(int),')')
            print( "Difference is significant, p-value ", np.round(ttest_,3))

1_reg 13 ( 20 )  vs  31 ( 21 )
Difference is significant, p-value  0.0


In [10]:
from statsmodels.stats.multitest import fdrcorrection
metric = 'Dice'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print('==== new data ===')
        fdr_pvalues_list = []
        keys_list = []
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key
            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                
                #assert if nan
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0
                
                data_1 = np.array(data_1)
                data_2 = np.array(data_2)

                ttost = ttost_paired(data_1, data_2, -data_2.std()/4, data_2.std()/4)[0]/7
                
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                fdr_pvalues_list.append(ttost)
                keys_list.append(key)

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if (ttest_ < 0.05)&(ttost > 0.05)::
                    print(data, model, metric)
                    print(preproc_1, preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                        (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                        ' vs ',\
                        (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                        (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttost,3))

        fdr_correction_mask = ~fdrcorrection(fdr_pvalues_list, alpha=0.05, method='indep', is_sorted=False)[0]

        print('FDR corrected p-values', np.take(list(keys_list),np.where(fdr_correction_mask==True)[0]))


SyntaxError: invalid syntax (2029292256.py, line 37)

In [ ]:
metric = 'Hausdorff'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print('==== new data ===')
        fdr_pvalues_list = []
        keys_list = []
        preproc_1 = "4a_resamp"

        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                
                #assert if nan
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0
                
                data_1 = np.array(data_1)
                data_2 = np.array(data_2)

                ttost = ttost_paired(data_1, data_2, -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7

                fdr_pvalues_list.append(ttost)
                keys_list.append(key)

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if (ttest_ < 0.05)&(ttost > 0.05)::
                    print(data, model, metric)
                    print(preproc_1,preproc_2, (np.round(data_1.mean(),3)*1).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*1).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*1).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*1).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttost,3))

        fdr_correction_mask = ~fdrcorrection(fdr_pvalues_list, alpha=0.05, method='indep', is_sorted=False)[0]

        print('FDR corrected p-values', np.take(list(keys_list),np.where(fdr_correction_mask==True)[0]))


=======================NEW MODEL=======================
==== new data ===
gbm nn_unet Hausdorff
4a_resamp 1_reg 13 ( 20 )  vs  31 ( 21 )
Difference is significant, p-value  0.143


gbm nn_unet Hausdorff
4a_resamp 3a_atlas 13 ( 20 )  vs  10 ( 11 )
Difference is significant, p-value  0.081
gbm nn_unet Hausdorff
4a_resamp 5_ss_shared 13 ( 20 )  vs  10 ( 14 )
Difference is significant, p-value  0.055
FDR corrected p-values ['1_reg' '3a_atlas' '5_ss_shared']
==== new data ===
bgpd nn_unet Hausdorff
4a_resamp 5_ss_shared 23 ( 34 )  vs  15 ( 17 )
Difference is significant, p-value  0.132
FDR corrected p-values ['1_reg' '4b_n4' '4d_susan' '6_hist' '5_ss_shared']
==== new data ===
lgg nn_unet Hausdorff
4a_resamp 2a_interp 38 ( 39 )  vs  29 ( 32 )
Difference is significant, p-value  0.085
lgg nn_unet Hausdorff
4a_resamp 3a_atlas 38 ( 39 )  vs  27 ( 35 )
Difference is significant, p-value  0.109
lgg nn_unet Hausdorff
4a_resamp 5_ss_shared 38 ( 39 )  vs  23 ( 24 )
Difference is significant, p-value  0.142
FDR corrected p-values ['2a_interp' '3a_atlas' '6_hist' '5_ss_shared']
=======================NEW MODEL=======================
==== new data ===
gbm UNETR Hausdorff
4a_resa

In [ ]:
metric = 'Surface Dice 1mm'
data = 'gbm'
label = 'WT'
model = 'nn_unet'
epoch = 300

for model in models:
    print('=======================NEW MODEL=======================')
    for data in df_1_3.Dataset.unique():
        print('==== new data ===')
        fdr_pvalues_list = []
        keys_list = []
        preproc_1 = "4a_resamp"
        for key in reindex_list:
            preproc_2 = key

            if key != "4a_resamp":

                data_1 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_1][metric]
                data_2 = df_1_3[df_1_3.Dataset==data][df_1_3.Epoch==epoch][df_1_3['Label (WT, ET, TC, GTV)']==label][df_1_3['NN-Architecture (nn-Unet, UNETR)']==model][df_1_3['Preprocessing step']==preproc_2][metric]
                
                #assert if nan
                assert data_1.isnull().sum() == 0
                assert data_2.isnull().sum() == 0
                
                data_1 = np.array(data_1)
                data_2 = np.array(data_2)

                ttost = ttost_paired(data_1, data_2, -data_2.std()/4, data_2.std()/4)[0]/7
                ttest_ = scipy.stats.wilcoxon(data_1, data_2,  alternative='two-sided')[1]*7
                
                fdr_pvalues_list.append(ttost)
                keys_list.append(key)

                # print("If more than 0.05", ttost > 0.05, np.round(ttost,3))
                if (ttest_ < 0.05)&(ttost > 0.05)::
                    print(data, model, metric)
                    print(preproc_2, (np.round(data_1.mean(),3)*100).round(0).astype(int), '(',
                      (np.round(data_1.std(),3)*100).round(0).astype(int), ')',
                      ' vs ',\
                      (np.round(data_2.mean(),3)*100).round(0).astype(int), '(' ,
                      (np.round(data_2.std(),3)*100).round(0).astype(int),')')
                    print( "Difference is significant, p-value ", np.round(ttest_,3))
        
        fdr_correction_mask = ~fdrcorrection(fdr_pvalues_list, alpha=0.05, method='indep', is_sorted=False)[0]

        print('FDR corrected p-values', np.take(list(keys_list),np.where(fdr_correction_mask==True)[0]))

=======================NEW MODEL=======================
==== new data ===
gbm nn_unet Surface Dice 1mm
1_reg 65 ( 20 )  vs  27 ( 15 )
Difference is significant, p-value  0.0
gbm nn_unet Surface Dice 1mm
4b_n4 65 ( 20 )  vs  60 ( 22 )
Difference is significant, p-value  0.0


FDR corrected p-values ['1_reg' '4b_n4']
==== new data ===
bgpd nn_unet Surface Dice 1mm
1_reg 43 ( 18 )  vs  20 ( 14 )
Difference is significant, p-value  0.0
FDR corrected p-values ['1_reg']
==== new data ===
lgg nn_unet Surface Dice 1mm
5_ss_shared 46 ( 25 )  vs  52 ( 23 )
Difference is significant, p-value  0.004
FDR corrected p-values ['5_ss_shared']
=======================NEW MODEL=======================
==== new data ===
gbm UNETR Surface Dice 1mm
1_reg 59 ( 21 )  vs  22 ( 12 )
Difference is significant, p-value  0.0
gbm UNETR Surface Dice 1mm
4b_n4 59 ( 21 )  vs  52 ( 21 )
Difference is significant, p-value  0.0
gbm UNETR Surface Dice 1mm
5_ss_shared 59 ( 21 )  vs  64 ( 20 )
Difference is significant, p-value  0.011
FDR corrected p-values ['1_reg' '4b_n4' '5_ss_shared']
==== new data ===
bgpd UNETR Surface Dice 1mm
1_reg 34 ( 16 )  vs  18 ( 13 )
Difference is significant, p-value  0.0
bgpd UNETR Surface Dice 1mm
4b_n4 34 ( 16 )  vs  40 ( 17 )
Difference is significant, p-value 